# Evreka Power BI Data Engineer Assessment Project

This notebook provides a comprehensive analysis of waste bin fullness data for the Evreka Data Engineer Assessment. The analysis includes data loading, cleaning, transformations, and insights into daily waste levels, collection times, and spatial distribution of waste bins.

---

### Key Objectives
1. **Data Preparation**: Clean and transform waste bin fullness data, capping overflow values and merging with location data.
2. **Analysis**:
   - **Daily Aggregation**: Calculate daily average fullness per bin location and waste type.
   - **Peak Collection Times**: Identify common collection hours.
   - **Geospatial Distribution**: Merge with latitude/longitude data for spatial analysis.

3. **Findings and Insights**:
   - **Daily Trends**: Observe trends in daily fullness levels.
   - **Collection Frequency**: Identify peak collection times to optimize schedules.
   - **Geospatial Analysis**: Determine areas with high waste bin utilization.



### Data Loading

In [18]:
import pandas as pd

# Load the datasets
waste_data = pd.read_csv('..\data\waste_data_timeseries\waste_data_timeseries.csv')
location_data = pd.read_csv('..\data\waste_data_timeseries\location_data.csv')

# Display first few rows of each dataset
waste_data.head(), location_data.head()


(   Unnamed: 0   SPID            VisitDate  # Paper  % Paper  # Plastic  \
 0           0  15801  2019-01-01 08:09:50        3     1.00          2   
 1           1  16160  2019-01-01 08:22:15        2     0.75          1   
 2           2   9960  2019-01-01 08:29:48        2     1.00          0   
 3           3  16135  2019-01-01 08:38:08        0     0.00          1   
 4           4  16017  2019-01-01 08:42:03        3     0.75          2   
 
    % Plastic  
 0       0.75  
 1       0.75  
 2       0.00  
 3       0.25  
 4       0.62  ,
    Unnamed: 0   SPID   Latitude  Longitude
 0           0  10222  39.933667  32.861833
 1           1  10374  39.923167  32.990167
 2           2  10375  39.922333  32.958000
 3           3  10377  39.917667  32.979167
 4           4  10912  39.990333  33.111000)

## Data Cleaning and Fullness Capping


In [19]:
# Convert 'VisitDate' to datetime format
waste_data['VisitDate'] = pd.to_datetime(waste_data['VisitDate'])

# Clip fullness values at 100% and identify overflow cases
fullness_columns = [col for col in waste_data.columns if col.startswith('%')]
for col in fullness_columns:
    waste_data[f"{col}_overflow"] = waste_data[col] > 100
    waste_data[col] = waste_data[col].clip(upper=100)

# Show cleaned data
waste_data[fullness_columns + [f"{col}_overflow" for col in fullness_columns]].head()



,% Paper,% Plastic,% Paper_overflow,% Plastic_overflow
0,1.00,0.75,False,False
1,0.75,0.75,False,False
2,1.00,0.00,False,False
3,0.00,0.25,False,False
4,0.75,0.62,False,False


## Daily Aggregation


In [20]:
# Extract date from VisitDate for daily aggregation
waste_data['date'] = waste_data['VisitDate'].dt.date

# Calculate daily average fullness per location
daily_avg_fullness = waste_data.groupby(['SPID', 'date'])[fullness_columns].mean().reset_index()

# Save daily average fullness data to CSV
daily_avg_fullness.to_csv('..\data\output_pbi\daily_avg_fullness.csv', index=False, sep=',', decimal=',')
print("daily_avg_fullness.csv saved successfully.")

# Display daily aggregated data
daily_avg_fullness.head()



daily_avg_fullness.csv saved successfully.


,SPID,date,% Paper,% Plastic
0,7444,2019-01-01,1.000000,0.000000
1,7444,2019-01-02,0.416667,0.333333
2,7444,2019-01-03,0.500000,0.375000
3,7444,2019-01-04,0.500000,0.126667
4,7444,2019-01-05,0.666667,0.233333


## Peak Collection Times


In [21]:
# Extract hour for analysis of collection times
waste_data['hour'] = waste_data['VisitDate'].dt.hour

# Count visits by hour to find peak collection times
peak_collection_times = waste_data.groupby(['SPID', 'hour']).size().reset_index(name='visit_count')

# Save peak collection times data to CSV
peak_collection_times.to_csv('..\data\output_pbi\peak_collection_times.csv', index=False, sep=',', decimal=',')
print("peak_collection_times.csv saved successfully.")

# Display peak collection times
peak_collection_times.head()



peak_collection_times.csv saved successfully.


,SPID,hour,visit_count
0,7444,7,14
1,7444,8,22
2,7444,9,49
3,7444,10,168
4,7444,11,187


## Geospatial Analysis


In [22]:
# Merge daily average fullness with location data
location_fullness = pd.merge(daily_avg_fullness, location_data[['SPID', 'Latitude', 'Longitude']], on='SPID', how='left')

# Save location fullness data to CSV
location_fullness.to_csv('..\data\output_pbi\location_fullness.csv', index=False, sep=',', decimal=',')
print("location_fullness.csv saved successfully.")

# Display merged data
location_fullness.head()


location_fullness.csv saved successfully.


,SPID,date,% Paper,% Plastic,Latitude,Longitude
0,7444,2019-01-01,1.000000,0.000000,39.941,33.056167
1,7444,2019-01-02,0.416667,0.333333,39.941,33.056167
2,7444,2019-01-03,0.500000,0.375000,39.941,33.056167
3,7444,2019-01-04,0.500000,0.126667,39.941,33.056167
4,7444,2019-01-05,0.666667,0.233333,39.941,33.056167


## Insights and Findings

### Key Insights
1. **Daily Waste Trends**: Daily averages show fluctuations in waste bin fullness levels, indicating peak days for waste collection.
2. **Collection Frequency**: Peak collection times are identified based on the `hour` column, helping optimize scheduling.
3. **Geospatial Patterns**: Geospatial data highlights areas with higher fullness levels, which may require more frequent collection services.

### Next Steps for Power BI Visualization
The following files can be used for Power BI integration to create interactive visuals:
- **`daily_avg_fullness.csv`**: For daily waste trends by location.
- **`peak_collection_times.csv`**: To analyze collection times.
- **`location_fullness.csv`**: For geospatial analysis and heatmap visualization.

